In [16]:
import pandas as pd

df = pd.read_csv("/kaggle/input/city-day/city_day.csv")
df.head()

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,01-01-2015,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,02-01-2015,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,03-01-2015,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,04-01-2015,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,05-01-2015,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN


In [17]:
data.isnull().sum()


City              0
Date              0
PM2.5          4594
PM10          11136
NO             3578
NO2            3581
NOx            4181
NH3           10324
CO             2055
SO2            3850
O3             4018
Benzene        5619
Toluene        8037
Xylene        18105
AQI            4677
AQI_Bucket     4677
dtype: int64

In [14]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29527 entries, 0 to 29526
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        29527 non-null  object 
 1   Date        29527 non-null  object 
 2   PM2.5       24933 non-null  float64
 3   PM10        18391 non-null  float64
 4   NO          25949 non-null  float64
 5   NO2         25946 non-null  float64
 6   NOx         25346 non-null  float64
 7   NH3         19203 non-null  float64
 8   CO          27472 non-null  float64
 9   SO2         25677 non-null  float64
 10  O3          25509 non-null  float64
 11  Benzene     23908 non-null  float64
 12  Toluene     21490 non-null  float64
 13  Xylene      11422 non-null  float64
 14  AQI         24850 non-null  float64
 15  AQI_Bucket  24850 non-null  object 
dtypes: float64(13), object(3)
memory usage: 3.6+ MB


In [19]:
df = df.drop(columns=[ 'NH3','City','Date','Toluene', 'Xylene', 'AQI_Bucket'], axis=1)


In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [21]:
columns_to_fill = ['PM2.5','PM10', 'NO', 'NO2', 'NOx', 'CO', 'SO2', 'O3', 'Benzene','AQI']

# Fill NaN values for the specified columns without affecting the Date column
df[columns_to_fill] = df[columns_to_fill].fillna(df[columns_to_fill].mean())

# Ensure the Date column remains intact
print(df.head()) 

       PM2.5        PM10     NO    NO2    NOx     CO    SO2      O3  Benzene  \
0  67.450578  118.127103   0.92  18.22  17.15   0.92  27.64  133.36     0.00   
1  67.450578  118.127103   0.97  15.69  16.46   0.97  24.55   34.06     3.68   
2  67.450578  118.127103  17.40  19.30  29.70  17.40  29.07   30.70     6.80   
3  67.450578  118.127103   1.70  18.48  17.97   1.70  18.59   36.08     4.43   
4  67.450578  118.127103  22.10  21.42  37.76  22.10  39.33   39.31     7.01   

          AQI  
0  166.463581  
1  166.463581  
2  166.463581  
3  166.463581  
4  166.463581  


In [26]:
air_quality_features = ['PM2.5','PM10', 'NO', 'NO2', 'NOx', 'CO', 'SO2', 'O3', 'Benzene']
x = df['AQI']
y = df[air_quality_features]


In [51]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Enable eager execution for easier debugging
tf.config.run_functions_eagerly(True)



# Check for and remove any infinite or NaN values
df = df.replace([np.inf, -np.inf], np.nan).dropna()

# Split the data into features and target
X = df[air_quality_features]
y = df['AQI']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the input features
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Normalize the target variable
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

# Print some statistics about the scaled data
print("X_train_scaled stats:")
print("Mean:", np.mean(X_train_scaled))
print("Std:", np.std(X_train_scaled))
print("Min:", np.min(X_train_scaled))
print("Max:", np.max(X_train_scaled))

print("\ny_train_scaled stats:")
print("Mean:", np.mean(y_train_scaled))
print("Std:", np.std(y_train_scaled))
print("Min:", np.min(y_train_scaled))
print("Max:", np.max(y_train_scaled))

X_train_scaled stats:
Mean: -2.7072878463405028e-17
Std: 1.0000000000000002
Min: -1.6953620553144246
Max: 30.937301036974382

y_train_scaled stats:
Mean: 1.0859232361432461e-16
Std: 1.0
Min: -1.1739107994519309
Max: 14.49029429992136
Epoch 1/100


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


591/591 ━━━━━━━━━━━━━━━━━━━━ 26s 43ms/step - loss: 0.3735 - mae: 0.3459 - val_loss: 0.1443 - val_mae: 0.2187
Epoch 2/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - loss: 0.1612 - mae: 0.2238 - val_loss: 0.1254 - val_mae: 0.2036
Epoch 3/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - loss: 0.1724 - mae: 0.2167 - val_loss: 0.1189 - val_mae: 0.1969
Epoch 4/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - loss: 0.1646 - mae: 0.2115 - val_loss: 0.1163 - val_mae: 0.1897
Epoch 5/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - loss: 0.1556 - mae: 0.2072 - val_loss: 0.1132 - val_mae: 0.1847
Epoch 6/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - loss: 0.1448 - mae: 0.1984 - val_loss: 0.1138 - val_mae: 0.1904
Epoch 7/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - loss: 0.1486 - mae: 0.1991 - val_loss: 0.1111 - val_mae: 0.1854
Epoch 8/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - loss: 0.1685 - mae: 0.2040 - val_loss: 0.1119 - val_mae: 0.1864
Epoch 9/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 25s

In [52]:
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(len(air_quality_features),)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# Learning rate scheduler
def lr_schedule(epoch):
    return 0.001 * 0.9 ** epoch

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Train the model
history = model.fit(
    X_train_scaled, y_train_scaled,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        keras.callbacks.LearningRateScheduler(lr_schedule)
    ],
    verbose=1
)

Epoch 1/100
  1/591 ━━━━━━━━━━━━━━━━━━━━ 50s 86ms/step - loss: 5.5092 - mae: 1.0746

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


591/591 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - loss: 0.6150 - mae: 0.3926 - val_loss: 0.1405 - val_mae: 0.2044 - learning_rate: 0.0010
Epoch 2/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - loss: 0.1831 - mae: 0.2200 - val_loss: 0.1322 - val_mae: 0.2096 - learning_rate: 9.0000e-04
Epoch 3/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - loss: 0.1811 - mae: 0.2131 - val_loss: 0.1188 - val_mae: 0.1950 - learning_rate: 8.1000e-04
Epoch 4/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - loss: 0.1562 - mae: 0.2056 - val_loss: 0.1176 - val_mae: 0.1910 - learning_rate: 7.2900e-04
Epoch 5/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - loss: 0.1609 - mae: 0.2029 - val_loss: 0.1165 - val_mae: 0.1912 - learning_rate: 6.5610e-04
Epoch 6/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - loss: 0.1498 - mae: 0.2001 - val_loss: 0.1179 - val_mae: 0.1879 - learning_rate: 5.9049e-04
Epoch 7/100
591/591 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - loss: 0.1759 - mae: 0.2037 - val_loss: 0.1160 - val_mae: 0.1849 -

In [54]:
model.save('/kaggle/working/model.h5')
print("done")


done


In [55]:


# Given input
user_input = pd.DataFrame({
    'PM2.5': [81],
    'PM10': [124],
    'NO': [1.44],
    'NO2': [20],
    'NOx': [12],
    'CO': [0.1],
    'SO2': [15],
    'O3': [127],
    'Benzene': [0.20]
})

# Function to predict AQI for new data
def predict_aqi(new_data):
    scaled_data = scaler_X.transform(new_data)
    scaled_predictions = model.predict(scaled_data)
    return scaler_y.inverse_transform(scaled_predictions)

# Function to interpret AQI values
def interpret_aqi(aqi):
    if aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Moderate"
    elif aqi <= 150:
        return "Unhealthy for Sensitive Groups"
    elif aqi <= 200:
        return "Unhealthy"
    elif aqi <= 300:
        return "Very Unhealthy"
    else:
        return "Hazardous"

# Make prediction
predicted_aqi = predict_aqi(user_input)

# Interpret the prediction
interpretation = interpret_aqi(predicted_aqi[0][0])

# Print results
print(f"Predicted AQI: {predicted_aqi[0][0]:.2f}")
print(f"AQI Category: {interpretation}")

# Feature importance analysis
feature_importance = np.abs(model.layers[0].get_weights()[0]).mean(axis=1)
feature_importance = 100 * feature_importance / np.sum(feature_importance)

# Print feature importance
print("\nFeature Importance:")
for feature, importance in zip(user_input.columns, feature_importance):
    print(f"{feature}: {importance:.2f}%")

# Compare input values to feature importance
print("\nInput values and their importance:")
for feature, value, importance in zip(user_input.columns, user_input.values[0], feature_importance):
    print(f"{feature}: {value} (Importance: {importance:.2f}%)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted AQI: 199.59
AQI Category: Unhealthy

Feature Importance:
PM2.5: 14.15%
PM10: 13.39%
NO: 9.85%
NO2: 11.37%
NOx: 11.17%
CO: 13.05%
SO2: 8.77%
O3: 7.89%
Benzene: 10.34%

Input values and their importance:
PM2.5: 81.0 (Importance: 14.15%)
PM10: 124.0 (Importance: 13.39%)
NO: 1.44 (Importance: 9.85%)
NO2: 20.0 (Importance: 11.37%)
NOx: 12.0 (Importance: 11.17%)
CO: 0.1 (Importance: 13.05%)
SO2: 15.0 (Importance: 8.77%)
O3: 127.0 (Importance: 7.89%)
Benzene: 0.2 (Importance: 10.34%)


/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
